In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot
import os
from copy import deepcopy

from time import time

from math import ceil
from scipy.stats import spearmanr, gamma, poisson

from anndata import AnnData, read_h5ad
import scanpy as sc
from scanpy import read
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset
from torch import tensor
from torch.cuda import is_available

from sciPENN.sciPENN_API import sciPENN_API

In [2]:
# read in data
adata_gene_train = sc.read("/home/asmauger/biostat625final/rna_train_hvg.h5ad")
adata_gene_test = sc.read("/home/asmauger/biostat625final/rna_test_hvg.h5ad")
adata_protein_train = sc.read("/home/asmauger/biostat625final/prot_train.h5ad")
adata_protein_test = sc.read("/home/asmauger/biostat625final/prot_test.h5ad")
ref = sc.read("/home/asmauger/biostat625final/pbmc_gene.h5ad")

In [4]:
# note that donor, day are stored as integers and might not work as expected with scipenn
# use 'daydonor' instead
# these data are already cell normalized and log normalized
sciPENN = sciPENN_API(gene_trainsets = [adata_gene_train], protein_trainsets = [adata_protein_train], 
                      gene_test = adata_gene_test, train_batchkeys = ['daydonor'], test_batchkey = 'daydonor',  use_gpu=False,
                     select_hvg=False, cell_normalize=False, log_normalize=False, min_cells=0, min_genes=0)

Using CPU

Normalizing Gene Training Data by Batch


100%|██████████| 9/9 [00:01<00:00,  6.69it/s]



Normalizing Protein Training Data by Batch


100%|██████████| 9/9 [00:00<00:00, 19.97it/s]



Normalizing Gene Testing Data by Batch


100%|██████████| 9/9 [00:01<00:00,  6.90it/s]


In [5]:
# make sure load=False, unless you want to re-use weights from a previous run
sciPENN.train(quantiles = [0.1, 0.25, 0.75, 0.9], n_epochs = 10000, ES_max = 12, decay_max = 6, 
             decay_step = 0.1, lr = 10**(-3), load = False)

Epoch 0 prediction loss = 1.412
Epoch 1 prediction loss = 0.957
Epoch 2 prediction loss = 0.951
Epoch 3 prediction loss = 0.951
Epoch 4 prediction loss = 0.949
Epoch 5 prediction loss = 0.944
Epoch 6 prediction loss = 0.941
Epoch 7 prediction loss = 0.941
Epoch 8 prediction loss = 0.940
Epoch 9 prediction loss = 0.939
Epoch 10 prediction loss = 0.937
Epoch 11 prediction loss = 0.938
Epoch 12 prediction loss = 0.942
Epoch 13 prediction loss = 0.943
Epoch 14 prediction loss = 0.938
Decaying loss to 0.0001
Epoch 15 prediction loss = 0.931
Epoch 16 prediction loss = 0.932
Epoch 17 prediction loss = 0.932
Epoch 18 prediction loss = 0.930
Epoch 19 prediction loss = 0.933
Epoch 20 prediction loss = 0.931
Decaying loss to 1e-05
Epoch 21 prediction loss = 0.931
Epoch 22 prediction loss = 0.931
Epoch 23 prediction loss = 0.932
Epoch 24 prediction loss = 0.932
Epoch 25 prediction loss = 0.931
Epoch 26 prediction loss = 0.931
Decaying loss to 1.0000000000000002e-06
Epoch 27 prediction loss = 0.931

In [6]:
imputed_test = sciPENN.predict()


In [7]:
embedding = sciPENN.embed()

In [8]:
embedding.X

array([[-0.07114612,  0.0996935 , -0.03793817, ...,  0.05308064,
         0.94526166, -0.11312702],
       [ 0.00309207, -0.45671356, -0.05833251, ...,  0.00453422,
         0.90640396,  0.31426886],
       [-0.04795591,  0.7697607 , -0.16859245, ...,  0.73876125,
        -0.99645233, -0.23429202],
       ...,
       [-0.0114318 ,  0.0395246 ,  0.04094888, ..., -0.06080035,
         0.9599461 , -0.06384736],
       [-0.05257633,  0.17112096,  0.2422376 , ..., -0.16921712,
        -0.9731447 , -0.03036799],
       [-0.05187283,  0.39211836,  0.07226393, ...,  0.55728424,
        -0.18512411,  0.29159755]], dtype=float32)

In [9]:
print(sum(adata_protein_test.obs.index == imputed_test.obs.index))
print(sum(imputed_test.var.index == adata_protein_test.var.index))

35492
134


In [10]:
adata_protein_test.X = adata_protein_test.X.toarray() 

adata_protein_test.layers['imputed'] = imputed_test.X
adata_protein_test.layers.update(imputed_test.layers)

# scaling by batch
batches = np.unique(adata_protein_test.obs['daydonor'].values)

for i in batches:
    indices = [x == i for x in adata_protein_test.obs['daydonor']]
    sub_adata = adata_protein_test[indices]

    sc.pp.scale(sub_adata)
    adata_protein_test[indices] = sub_adata.X


/home/asmauger/.conda/envs/scipenn_env/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [11]:
MSEs= ((adata_protein_test.X - adata_protein_test.layers["imputed"])**2).mean(axis = 0)**(1/2)

In [12]:
print(MSEs)

[0.9434334  0.93891424 0.8272301  0.6837612  0.49599466 0.57999104
 0.54665387 0.9445831  0.774013   0.8237878  0.96721596 1.0007776
 0.9229987  0.98553693 0.75786525 0.9315148  0.5729405  0.52814025
 0.69198567 0.9973349  0.70513374 0.7361803  0.9941762  0.93652934
 0.53719866 0.9760478  0.9912728  0.9606974  0.79983413 0.76370925
 0.8897634  0.8981121  0.97100025 0.52838653 0.9471252  0.98424655
 0.91028076 0.993705   0.9498395  0.6646052  0.94207007 0.8739161
 0.9962819  0.9293417  0.4409494  0.92751443 0.83089346 0.89240074
 0.9484363  0.9934209  0.94460136 0.5996777  0.9447781  0.62315583
 0.9879651  0.9889469  0.9374008  0.7034933  0.87581915 0.860426
 0.938921   0.98949534 0.99778444 0.94306314 0.8509973  0.9416636
 0.676276   0.6874367  0.9971843  0.5455326  0.8301205  0.5389459
 0.98276013 0.92263263 0.9906135  0.964939   0.7372445  0.9516104
 0.8125406  0.9697533  0.86225796 0.9402132  0.90905654 0.5789137
 0.92603976 0.8031253  0.6955167  0.8884019  0.58652925 0.9942137
 0.9